In [1]:
# Part 1: Import Libraries and Define the Cage Class
import copy
import random
import time
import math

class Cage:
    def __init__(self, cells, operation, target):
        self.cells = cells
        self.operation = operation
        self.target = target
        self.so_far_calculated = 0
        if operation == '*' or operation == '/':
            self.so_far_calculated = 1

In [2]:
# Part 2: Function to Generate KenKen Puzzle

def generate_KenKen(size):
    grid = [[0]*size for _ in range(size)]
    fill_grid_with_numbers(grid , size)
    cages = generate_random_cages(grid , size)
    return grid , cages

# Part 3: Function to Fill Grid with Numbers

def fill_grid_with_numbers(grid, size):
    """
    This function fills the grid with numbers 1 to size such that
    each row and each column has unique values. It uses backtracking
    to ensure all cells are filled with valid numbers.
    """

    def is_safe_to_place(grid, row, col, num):
        # Check row and column uniqueness
        for i in range(0 , size):
            if grid[row][i] == num:
                return False
            if grid[i][col] == num:
                return False
        return True

    def fill_grid_backtracking(grid, row, col):
        # Fills the grid recursively with valid numbers using backtracking
        
        if col == size:
            col = 0
            row +=1
        if row == size:
            return True
        arr = [x+1 for x in range(size)]
        random.shuffle(arr)
        for i in arr:
            if is_safe_to_place(grid , row , col , i):
                grid[row][col] = i
                if fill_grid_backtracking(grid , row, col + 1):
                    return True
                else:
                    grid[row][col] = 0
        return False
    
    # Start backtracking from the first cell
    fill_grid_backtracking(grid, 0, 0)



# Part 4: Function to Generate Random Cages

def generate_random_cages(grid, size):
    cages = []
    visited = [[False] * size for _ in range(size)]

    for i in range(size):
        for j in range(size):
            if not visited[i][j]:
                cage_size = random.randint(1, size)
                cells = [(i, j)]
                visited[i][j] = True

                while len(cells) < cage_size:
                    x, y = cells[-1]
                    neighbors = [(x + dx, y + dy) for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]
                                 if 0 <= x + dx < size and 0 <= y + dy < size and not visited[x + dx][y + dy]]
                    if neighbors:
                        next_cell = random.choice(neighbors)
                        cells.append(next_cell)
                        visited[next_cell[0]][next_cell[1]] = True
                    else:
                        break

                operation = random.choice(['+', '*']) if cage_size > 2 else random.choice(['+', '-', '*', '/'])
                target = calculate_target(grid, cells, operation)
                cages.append(Cage(cells, operation, target))

    return cages


# Function to calculate the target based on the operation for a given cage
def calculate_target(grid, cells, operation):
    if operation == '*':
        res = 1
        for (row , col) in cells:
            res *= grid[row][col]
        return res
    if operation == '+':
        res = 0
        for (row , col) in cells:
            res += grid[row][col]
        return res
    if operation == '-':
        res = -20
        for (row , col) in cells:
            if res == -20:
                res = grid[row][col]
            else:
                res = abs(res - grid[row][col])
        return res
    if operation == '/':
        res = grid[cells[0][0]][cells[0][1]]
        if len(cells) > 1:
            res = max(grid[cells[0][0]][cells[0][1]] , grid[cells[1][0]][cells[1][1]]) \
            / min (grid[cells[0][0]][cells[0][1]] , grid[cells[1][0]][cells[1][1]])
            #fuck my life
        return res




In [3]:
class Heap:
    def __init__(self, comparer, data=None):
        self.comparer = comparer
        self.list = []
        self.pointer = {}
        print(data)
        for i in range (len (data)):
            self.pointer[data[i][0]] = i
        if data:
            self.list = data[:]
            for i in range(len(self.list) // 2, -1, -1):
                self.sift_down(i)

    def get_top(self):
        self.check_exception()
        return self.list[0]

    def pop_top(self):
        self.check_exception()
        self.pointer[self.list[0][0]] = -1
        self.list[0] = self.list[-1]
        self.pointer[self.list[-1][0]] = 0
        self.list.pop()
        if len (self.list) ==0:
            return
        self.sift_down(0)

    def insert(self, value):
        #print (f"inserted element {value[0]} with prio {value[1]} in heap")
        self.pointer[value[0]] = len(self.list)
        self.list.append(value)
        self.sift_up(len(self.list) - 1)

    def print_list(self):
        print(" ".join(map(str, self.list)))

    def size(self):
        return len(self.list)

    def check_exception(self):
        #return
        if len(self.list) == 0:
            raise Exception("Heap is empty")

    def swap(self, i1, i2):
        self.pointer[self.list[i1][0]] , self.pointer[self.list[i2][0]] = self.pointer[self.list[i2][0]] , self.pointer[self.list[i1][0]] 
        self.list[i1], self.list[i2] = self.list[i2], self.list[i1]

    def sift_down(self, index):
        left = index * 2 + 1
        right = index * 2 + 2

        if left >= len(self.list):
            return

        if right >= len(self.list):
            target_index = left
        else:
            target_index = left if self.comparer(self.list[left], self.list[right]) else right

        if self.comparer(self.list[target_index], self.list[index]):
            self.swap(target_index, index)
            self.sift_down(target_index)

    def sift_up(self, index):
        if index == 0:
            return

        parent = (index - 1) // 2
        if self.comparer(self.list[index], self.list[parent]):
            self.swap(parent, index)
            self.sift_up(parent)
    def update(self , key , value):
        if len (self.list) <= self.pointer.get(key):
            return
        
        self.list[self.pointer.get(key)] = (self.list[self.pointer.get(key)][0] , self.list[self.pointer.get(key)][1] + value)
        self.sift_up(self.pointer.get(key))
        self.sift_down(self.pointer.get(key))
    
    def update_replace(self , key , value):
        if (len (self.list) <= self.pointer.get(key)) or (self.pointer[key] == -1):
            return
        
        self.list[self.pointer[key]] = (self.list[self.pointer[key]][0] , value)
        self.sift_up(self.pointer[key])
        self.sift_down(self.pointer[key])

    def heap_check(self):
        for i in range (len(self.list)):
            if i!= self.pointer[self.list[i][0]] :
                print (f"{self.list[i][0]} is in position {i} and pointer has index {self.pointer[self.list[i][0]]} and prio is {self.list[i][1]}")
            #print (f"{self.list[i][0]} is in position {i} and pointer has index {self.pointer[self.list[i][0]]} and prio is {self.list[i][1]}")
            
def comp(a, b):
    return a[1] < b[1]


In [4]:
# Part 5: KenKen Backtracking Solver

def solve_kenken(grid, cages):
    x , y = find_unassigned_location(grid)
    #x , y = find_unassigned_location(grid)
    if x == -1:
        return True
    arr = [x+1 for x in range(len(grid))]
    random.shuffle(arr)
    for i in  arr:
        grid[x][y] = i
        if is_safe_kenken(grid , x , y , i , cages):
            #print("it is safe for " + x +" " + y + " for value " + i)
            #grid[x][y] = i
            if solve_kenken(grid , cages):
                return True
            else:
                grid[x][y] = 0
        else:
            grid[x][y] = 0      
    return False

def is_safe_kenken(grid, row, col, num, cages):
    # Check row, column and cage numbers uniqueness
    for i in range(len(grid)):
        if i!=col and grid[row][col] == grid[row][i]:
            return False
        
    for i in range(len(grid)):
        if i!=row and grid[row][col] == grid[i][col]:
            return False
    for cage in cages:
        if (row , col) in cage.cells : 
            return validate_cage_operation( grid , cage)


def validate_cage_operation( grid , cage):
    # Check if the operation on the cage values matches the target
    res = 0
    if cage.operation == '*' or cage.operation == '/':
        res = 1
    for cell in cage.cells:
        if grid[cell[0]][cell[1]] == 0:
            return True
        if (cage.operation == '*'):
            res *= grid[cell[0]][cell[1]]
        elif cage.operation == '+':
            res += grid[cell[0]][cell[1]]
        elif cage.operation == '-':
            res =abs (res - grid[cell[0]][cell[1]])
        elif cage.operation == '/':
            res = max(res , grid[cell[0]][cell[1]]) / min (res , grid[cell[0]][cell[1]])
    return cage.target == res    


def find_unassigned_location(grid):
    # if i + 1 == len(grid):
    #     if j + 1 == len(grid):
    #         return -1, -1
    #     return 0 , j + 1
    # return i + 1 , j
    # Find the first empty cell in the grid
    for i in range(len(grid)):
        for j in range (len(grid[i])):
            if grid[i][j] == 0:
                return i , j
    return -1 , -1



In [5]:
# Part 7: Print Solution

def print_solution(grid):
    for row in grid:
        print(" ".join(str(x) for x in row))


In [42]:
# Part 6: KenKen Domain Constraint Solver

def solve_kenken_csp(grid, cages):
    """
    Function to solve the KenKen grid using Constraint Satisfaction Problem (CSP)
    """

    def create_domains():
        """
        Function to create domains for each cell in the grid
        """
        mydic = {}
        all_possibilities = set([i+1 for i in range(len(grid))])
        for i in range(len(grid)):
            for j in range(len(grid)):
                mydic[(i , j)] = copy.copy(all_possibilities)
        for cage in cages:
            if len(cage.cells) == 1:
                    mydic[(cage.cells[0][0] , cage.cells[0][1])] = set([int(cage.target)])
            else:                
                #if cage.operation=='-':
                if cage.operation == '*':
                    temp = set()
                    for i in range(1 , len(grid)+1):
                        if cage.target %i == 0:
                            temp.add(i)
                    
                    for cell in cage.cells:
                        mydic[(cell[0] , cell[1])] = temp.copy()
                
        #print (mydic)
        return mydic

    def is_valid_assignment(i, j, val):
        """
        Function to check if assigning a value to a cell is valid
        """
        thecage = None
        prev_sodar = 0
        for cage in cages:
            if (i , j) in cage.cells :
                thecage = cage
                prev_sodar = cage.so_far_calculated
                grid[i][j] = val
                #print (f"calculated {cage.so_far_calculated} for cage {cage} for [{i}][{j}]")
                valid , prev_versions = validate_cage_operation(cage , i , j)
                grid[i][j] = 0
                if not valid : 
                    #print (f"failed at cage {val} with pos [{i}][{j}] ")
                    return valid , prev_versions
                break
        for ii in range(len(grid)):
            if ii!=j and len(assignment[(i , ii)]) == 1 and val in assignment[(i , ii)]:
                if thecage != None:
                    thecage.so_far_calculated = prev_sodar
                return False , prev_versions
        
        for ii in range(len(grid)):
            if ii!=i and len(assignment[(ii , j)]) == 1 and val in assignment[(ii , j)]:
                if thecage != None:
                    thecage.so_far_calculated = prev_sodar
                return False , prev_versions
        return True , prev_versions
    
    def validate_cage_operation(cage , x , y):
        
    # Check if the operation on the cage values matches the target
        prev_version = {}
        if (cage.operation == '/'):
            a = grid[cage.cells[0][0]][cage.cells[0][1]]
            if (len(cage.cells) == 1):
                return cage.target == a , prev_version
            b = grid[cage.cells[1][0]][cage.cells[1][1]]
            if a==0:
                a_pos = (cage.cells[0][0] , cage.cells[0][1])
                temp = set([cage.target*b , b /cage.target ])
                temp = temp.intersection(assignment[a_pos])
                if len(temp) == 0:
                    return False , prev_version
                else:
                    prev_version[a_pos] = assignment[a_pos]
                    assignment[a_pos] = temp
                    myheap.update_replace(a_pos , len(assignment[a_pos]))
            elif b==0:
                b_pos = (cage.cells[1][0] , cage.cells[1][1])
                temp = set([cage.target*a , a /cage.target ])
                #print (assignment[b_pos])
                temp = temp.intersection(assignment[b_pos])
                if len(temp) == 0:
                    return False , prev_version
                else:
                    #print (temp)
                    prev_version[b_pos] = assignment[b_pos]
                    assignment[b_pos] = temp
                    myheap.update_replace(b_pos , len(assignment[b_pos]))
        
        elif cage.operation =='-':
            a = grid[cage.cells[0][0]][cage.cells[0][1]]
            if (len(cage.cells) == 1):
                return cage.target == a , prev_version
            b = grid[cage.cells[1][0]][cage.cells[1][1]]
            if a==0:
                a_pos = (cage.cells[0][0] , cage.cells[0][1])
                temp = set([cage.target + b , abs (cage.target - b) ])
                temp = temp.intersection(assignment[a_pos])
                if len(temp) == 0:
                    return False , prev_version
                else:
                    prev_version[a_pos] = assignment[a_pos]
                    assignment[a_pos] = temp
                    myheap.update_replace(a_pos , len(assignment[a_pos]))
                
            elif b==0:
                b_pos = (cage.cells[1][0] , cage.cells[1][1])
                temp = set([abs(cage.target - a) , a + cage.target ])
                temp = temp.intersection(assignment[b_pos])
                if len(temp) == 0:
                    return False , prev_version
                else:
                    prev_version[b_pos] = assignment[b_pos]
                    assignment[b_pos] = temp
                    myheap.update_replace(b_pos , len(assignment[b_pos]))
        
        elif (cage.operation =='*'):
            if(cage.target % (cage.so_far_calculated * grid[x][y]) != 0):
                return False , prev_version
            cage.so_far_calculated *= grid[x][y]
            allowed = cage.target / cage.so_far_calculated
            changes = []
            for cell in cage.cells:
                if grid[cell[0]][cell[1]] == 0:
                    temp = set()
                    for dom in assignment[(cell[0] , cell[1])]:
                        if allowed % dom == 0:
                            temp.add(dom)
                    if len(temp) == 0:
                        cage.so_far_calculated /= grid[x][y]
                        return False , prev_version
                    changes.append(temp)
                    
            counter = 0
            for cell in cage.cells:
                if grid[cell[0]][cell[1]] == 0:
                    prev_version [(cell[0] , cell[1])] = assignment[(cell[0] , cell[1])]
                    assignment[(cell[0] , cell[1])] = changes[counter]
                    myheap.update_replace((cell[0] , cell[1]) , len(changes[counter]))
                    counter +=1
        
        elif cage.operation =='+':
            cage.so_far_calculated += grid[x][y]
            changes = []
            flag = False
            temp = cage_restriction_for_plus(cage)
            for cell in cage.cells:
                if grid [cell[0]][cell[1]] == 0:
                    flag = True
                    #temp = set([i for i in range(1 ,math.floor (cage.target - cage.so_far_calculated + 1))])
                    #print ("haha this is the place:")
                    #print(temp)
                    #print(temp)
                    #print ("******")
                    tempp = temp.intersection(assignment[cell[0] , cell[1]])
                    if len(tempp) == 0:
                        cage.so_far_calculated -= grid[x][y]
                        return False , prev_version
                    changes.append(tempp)
            if not flag and cage.so_far_calculated != cage.target:
                cage.so_far_calculated -= grid[x][y]
                return False , prev_version
                
            counter = 0
            
            for cell in cage.cells:
                if grid[cell[0]][cell[1]] == 0:
                    prev_version [(cell[0] , cell[1])] = assignment[(cell[0] , cell[1])]
                    assignment[(cell[0] , cell[1])] = changes[counter]
                    myheap.update_replace((cell[0] , cell[1]) , len(changes[counter]))
                    counter +=1
        return True , prev_version
    def cage_restriction_for_plus(cage):
        empty_cell = max_sums = min_sums = minmax = maxmin = 0
        minmax = 1000000
        maxmin = -1 * minmax
        for cell in cage.cells:
            if grid[cell[0]][cell[1]] == 0:
                minval = min (assignment[cell])
                maxval = max (assignment[cell])
                min_sums += minval
                max_sums += maxval
                minmax = min(minmax , maxval)
                maxmin = max (maxmin , minval)
                empty_cell +=1
        #print (f" plus restriction log : {max_sums} , {minmax} , {min_sums} , {maxmin}")
        return set([i for i in range(math.floor(max( (cage.target - cage.so_far_calculated) - (max_sums - minmax) , 1)) ,math.floor( min((cage.target - cage.so_far_calculated) - (min_sums - maxmin), min (len(grid) , (cage.target - cage.so_far_calculated))) + 1 ))])
        
        
    def find_unassigned_location():
        """
        Function to find an unassigned location in the grid 
        """
        if len(myheap.list) == 0:
            return -1 , -1
        pos = myheap.get_top()
        myheap.pop_top()
        #print(f"selected {pos[0]} with prio {pos[1]}")
        if pos[1]!= len(assignment[pos[0]]):
            
            print(f"selected {pos[0]} with prio {pos[1]} and actuel prio of {len(assignment[pos[0]])}")
        return pos[0]

    def remove_assignments( x , y ):
        heha =[]
        for i in range(len(grid)):
            if i!=x and grid[x][y] in assignment[(i , y)]:
                assignment[(i , y)].remove(grid[x][y])
                heha.append((i , y))
                myheap.update_replace((i , y) , len(assignment[(i , y)]))
        
        for i in range(len(grid)):
            if i!=y and grid[x][y] in assignment[(x , i)]:
                assignment[(x , i)].remove(grid[x][y])
                heha.append((x , i))
                myheap.update_replace((x , i) , len(assignment[(x , i)]))
        return heha
    def add_assignments( heha , value):
        for (x , y) in heha:
            assignment[(x , y)].add(value)
            myheap.update_replace((x , y) , len (assignment[(x , y)]))
        
    def solve_csp():
        """
        Recursive function to solve grid with CSP
        """
        #print_solution(grid)
        #print (assignment)
        #print("-----------------------")
        #myheap.heap_check()
        i , j = find_unassigned_location()
        if i == -1:
            return True
        haha = assignment[(i , j)].copy()
        for possib in haha:
            #print(f"trying {possib} for pos [{i}][{j}]")
            valid , prev_versions = is_valid_assignment(i , j , possib) 
            if valid:
                grid[i][j] = possib
                heha = remove_assignments(i , j)
                if solve_csp():
                    return True
                else:           
                    add_assignments(heha , grid[i][j])
                    for key , value in prev_versions.items():
                        assignment[key] = value
                        myheap.update_replace (key , len(value))
                    roll_back_cage(i , j)
                    grid[i][j] = 0
            else:
                for key , value in prev_versions.items():
                    assignment[key] = value
                    myheap.update_replace (key , len(value))
                
        assignment[(i , j)] = haha
        myheap.insert(((i , j) , len(haha)))
        
        #print(f"failed for pos [{i}][{j}]")
        return False

    def roll_back_cage(i , j):
        for cage in cages:
            if (i, j) in cage.cells:
                if (cage.operation == '*'):
                    cage.so_far_calculated /= grid[i][j]
                elif cage.operation == '+':
                    cage.so_far_calculated -= grid[i][j]
                 
        


    # Create initial domains for each cell in the grid
    domains = create_domains()
    assignment = {(i, j): val for (i, j), val in domains.items()}
    arrr = [(key, len(val)) for key , val in assignment.items()]
    myheap = Heap(comp , arrr)
    # Solve the KenKen grid using CSP
    if solve_csp():
        solved_grid = [[assignment[(i, j)] for j in range(len(grid))] for i in range(len(grid))]
        return solved_grid
    else:
        return None


In [70]:
size = 7
solved_grid, kenken_cages = generate_KenKen(size)

print("Generated KenKen Cages:")
for cage in kenken_cages:
    print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target}")

# Solve using Backtracking solver
backtrack_grid = [[0]*size for _ in range(size)]
unsolved_grid = [[0]*size for _ in range(size)]

a = time.time()
if solve_kenken(backtrack_grid, kenken_cages):
    print("Solved KenKen Puzzle (BackTracking):")
    print_solution(backtrack_grid)
else:
    print("No solution found using BackTracking.")
print("-----------------------")
print (time.time() - a)
# Solve using domain CSP solver

Generated KenKen Cages:
Cage Cells: [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1)], Operation: +, Target: 17
Cage Cells: [(0, 1), (0, 2)], Operation: /, Target: 3.0
Cage Cells: [(0, 3), (0, 4), (0, 5)], Operation: *, Target: 20
Cage Cells: [(1, 1), (1, 2), (2, 2)], Operation: *, Target: 36
Cage Cells: [(1, 3), (2, 3), (3, 3)], Operation: +, Target: 12
Cage Cells: [(1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4)], Operation: *, Target: 108
Cage Cells: [(1, 5)], Operation: +, Target: 4
Cage Cells: [(3, 0), (4, 0), (4, 1), (5, 1)], Operation: +, Target: 13
Cage Cells: [(3, 2), (4, 2), (4, 3), (5, 3), (5, 2)], Operation: +, Target: 18
Cage Cells: [(3, 4)], Operation: +, Target: 5
Cage Cells: [(5, 0)], Operation: +, Target: 4
Cage Cells: [(5, 4), (5, 5)], Operation: *, Target: 12
Solved KenKen Puzzle (BackTracking):
3 2 6 1 4 5
5 6 3 2 1 4
1 5 2 4 3 6
2 3 4 6 5 1
6 4 1 5 2 3
4 1 5 3 6 2
-----------------------
0.03337597846984863


In [71]:
# Part 8: Run Example
a = time.time()
#idk why but this is neccessary
for cage in kenken_cages:
    if cage.operation == '*' or cage.operation == '/':
        cage.so_far_calculated = 1
    else:
        cage.so_far_calculated =0
#for cage in kenken_cages:
#    print(f"Cage Cells: {cage.cells}, Operation: {cage.operation}, Target: {cage.target} , so far : {cage.so_far_calculated}")
unsolved_grid = [[0]*size for _ in range(size)]
print_solution(unsolved_grid)
csp_solved= solve_kenken_csp(unsolved_grid, kenken_cages)
if csp_solved:
    print("Solved KenKen Puzzle (domain CSP):")
    print_solution(csp_solved)
else:
    print("No solution found using CSP.")

print(time.time() - a)

0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
[((0, 0), 6), ((0, 1), 6), ((0, 2), 6), ((0, 3), 4), ((0, 4), 4), ((0, 5), 4), ((1, 0), 6), ((1, 1), 5), ((1, 2), 5), ((1, 3), 6), ((1, 4), 5), ((1, 5), 1), ((2, 0), 6), ((2, 1), 6), ((2, 2), 5), ((2, 3), 6), ((2, 4), 5), ((2, 5), 5), ((3, 0), 6), ((3, 1), 6), ((3, 2), 6), ((3, 3), 6), ((3, 4), 1), ((3, 5), 5), ((4, 0), 6), ((4, 1), 6), ((4, 2), 6), ((4, 3), 6), ((4, 4), 5), ((4, 5), 5), ((5, 0), 1), ((5, 1), 6), ((5, 2), 6), ((5, 3), 6), ((5, 4), 5), ((5, 5), 5)]
Solved KenKen Puzzle (domain CSP):
{3} {2} {6} {1} {4} {5}
{5} {6} {3} {2} {1} {4}
{1} {5} {2} {4} {6} {3}
{2} {3} {4} {6} {5} {1}
{6} {4} {1} {5} {3} {2}
{4} {1} {5} {3} {2} {6}
0.029439687728881836
